In [100]:
from typing import Iterable

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from scipy.linalg import expm

def random_unitary_iter(r2:int):
    r1 = 1
    rham = np.random.rand(2**r2,2**r2) + 1j*np.random.rand(2**r2,2**r2)
    rham += np.conj(np.transpose(rham))
    irham = 1j*rham
    rut=expm(irham)
    rut/np.trace(rut)
    return rut

def controlled_U_gate(circ:QuantumCircuit, ctrl:int, targs:list, rut:np.ndarray)->None:
    rut_gate=UnitaryGate(rut)
    crut_gate = rut_gate.control(1)
    circ.append(crut_gate,qargs=[ctrl,*targs])


def sample_circuit(circuit: QuantumCircuit) -> dict:                #Von mir um den Circuit zu messen
  n_shots = 1
  statevector = Statevector.from_instruction(circuit)
  outcomes = statevector.sample_counts(n_shots)
  return outcomes


In [101]:
from typing import Tuple

from qiskit import QuantumRegister
from qiskit.result import Counts
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt

def plot_counts(counts:Counts):
    plt.plot(
        range(len(counts.int_outcomes().values())),
        np.abs(list(counts.int_outcomes().values())),
        "rs:",
        markerfacecolor='none',
        label="counts"
    )
    plt.legend()
    plt.show()

def get_random_phase_and_psi(unitary:np.ndarray)->np.ndarray:
    eigvals,eigvecs=np.linalg.eig(unitary)
    idx = np.random.randint(0,len(eigvals))

    # eigval = e^{2pi i theta}
    eigval = eigvals[idx]
    phase = np.real(-1j * np.log(eigval)/(2*np.pi))
    phase = np.mod(phase,1)
    psi = eigvecs[:,idx]
    return phase,psi

def qpe_circuit(r2:int, omega, outcome, k:int)->Tuple[QuantumCircuit,float]:
    r1 = 1
    first_register=QuantumRegister(r1,"first")
    second_register=QuantumRegister(r2,"second")
    third_register=QuantumRegister(1,"third")               #das wird der control qubit für die rotation, der speichert das vorherige Ergebnis ab

    circ = QuantumCircuit(first_register,second_register,third_register)
    circ.h(first_register)
    if outcome == 1:
        circ.x(third_register)

    unitary = random_unitary_iter(r2)

    get_phase_and_psi=True
    if get_phase_and_psi:
        phase,psi= get_random_phase_and_psi(unitary)
        get_phase_and_psi=False

    for l in range(2**k):
        controlled_U_gate(circ=circ,ctrl=first_register,targs=second_register,rut=unitary)

    #circ.append(QFT(num_qubits=len(first_register),inverse=True,do_swaps=True),qargs=first_register)
    circ.crz(omega, third_register, first_register)
    circ.h(first_register)

    return circ,phase,psi

def bitstring_idx_to_phase(index)->float:
    bin(index)

def get_phase_from_qpe_circuit(circ:QuantumCircuit,psi:np.ndarray)->float:

    out_circ = circ.copy_empty_like()

    out_circ.prepare_state(state=psi,qubits=circ.qregs[1])
    out_circ.append(circ,qargs=circ.qubits)
    statevector = Statevector.from_instruction(out_circ)

    r1=len(out_circ.qregs[0])
    counts=statevector.sample_counts(1000,qargs=range(r1))
    bitstring =counts.most_frequent()

    return np.sum([(int(v))*2**-(i+1) for i,v in enumerate(bitstring)])

In [103]:
outcome = 420               #irgend ne random zahl
for trial in range(3):                         #in trial lasse ich den circuit einfach mehrmals für verscheiden unitaries laufen
    real_phase = 0
    for loop in range(3):                      #in loop mache ich 10 durchläufe um eine phase zu bestimmen
        if loop == 0:
            outcome = 1
            omega = 0
        circ,phase,psi = qpe_circuit(3, omega, outcome, loop)
        computed_phase = get_phase_from_qpe_circuit(circ,psi)
        omega = computed_phase
        real_phase += omega
        x = sample_circuit(circ)
        #print_outcomes(outcomes)
        outcome=420
        for bitstring,counts in x.items():
            outcome = max({counts})
    print(f"trial {trial} | correct: {phase:.5f}, guess: {computed_phase:.5f} err: {np.abs(phase-computed_phase):.5f}")

trial 0 | correct: 0.24428, guess: 0.00000 err: 0.24428
trial 1 | correct: 0.11288, guess: 0.50000 err: 0.38712
trial 2 | correct: 0.96370, guess: 0.00000 err: 0.96370


In [ ]:
x = sample_circuit(circ)
#print_outcomes(outcomes)
outcome=420
for bitstring,counts in x.items():
   outcome = max({counts})
print(outcome)

1
